In [1]:
from safe_control_gym.envs.manipulators.manipulator import BaseManipulator
import numpy as np 
import random 
from time import sleep

pybullet build time: Mar 29 2022 22:56:14


In [2]:
urdf_path = "../safe_control_gym/envs/manipulators/assets/franka_panda/panda.urdf"
control_mode = "torque"
target_space = "joint"

env = BaseManipulator(
    urdf_path,
    control_mode,
    target_space 
)

argv[0]=
Version = 4.1 Metal - 76.3
Vendor = Apple
Renderer = Apple M1 Pro
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [3]:
env.n_joints

13

In [23]:
link_states =  env._pb_client.getLinkState(env.robot,
                                            linkIndex=6,
                                            computeLinkVelocity=True)


In [24]:
link_states

((0.039833163452802724, -0.005246717093960421, 1.0411480839438747),
 (0.6968757615251299,
  -0.11944479553082608,
  0.025003790002564637,
  0.7067332766361558),
 (0.04, 0.0, 0.0),
 (0.0, 0.0, 0.0, 1.0),
 (0.0010245442390441895, -1.3434328138828278e-06, 1.0330008268356323),
 (0.6968757510185242,
  -0.11944480240345001,
  0.025003790855407715,
  0.7067332863807678),
 (0.16140478353054072, -0.021992152304630256, -0.7820516002624263),
 (2.675834999501902, 19.791254360890452, -0.004319621550829485))

In [25]:
# only extract world positions
link_states_dict = {
    "position": link_states[4],
    "orientation": link_states[5],
    "linear_vel": link_states[6],
    "angular_vel": link_states[7]
}

In [26]:
link_states_dict

{'position': (0.0010245442390441895,
  -1.3434328138828278e-06,
  1.0330008268356323),
 'orientation': (0.6968757510185242,
  -0.11944480240345001,
  0.025003790855407715,
  0.7067332863807678),
 'linear_vel': (0.16140478353054072,
  -0.021992152304630256,
  -0.7820516002624263),
 'angular_vel': (2.675834999501902, 19.791254360890452, -0.004319621550829485)}

In [26]:
action_space = [ -200, -300, -400, 200, 300, 400,]


for i in range(50):
#     action_index = env.n_joints-2
    action_index = 6
    action_list = np.zeros(env.n_joints)
    action_list[action_index] = random.choice(action_space)    
    obs,reward, done,info = env.step(action_list)
#     if i%5== 0 :
#           print(f"position: {obs[action_index][5]}") # link position
#           print(f"angular velocity: {obs[action_index][-1]}") # velocity
                
    sleep(0.05)
    
# for i in range(50):
#     #     action_index = env.n_joints-2
#     action_index = 6
#     action_list = np.zeros(env.n_joints)
#     action_list[action_index] = random.choice(action_space)    
#     obs,reward, done,info = env.step(action_list)
# #     if i%5== 0 :
# #           print(f"position: {obs[action_index][5]}") # link position
# #           print(f"angular velocity: {obs[action_index][-1]}") # velocity
                
#     sleep(0.05)

In [23]:
from gym import spaces 
# env.observation_space = spaces.Box(-np.inf, np.inf, (14,), dtype=np.float32)

# orientation use quaternion so 4 dim 

# action_space = [spaces.Discrete(1) for i in range(env.n_joints-1)]
# action_space.append(spaces.Box(-400, 400, shape=(1,), dtype=np.float32))

# env.action_space = spaces.Tuple(
#     action_space
# )
env.action_space = spaces.Box(300, 400, (env.n_joints,), dtype=np.float32)
env.observation_space = spaces.Box(-np.inf, np.inf, (4,), dtype=np.float32)

# env.observation_space = spaces.Dict({
#       'position': spaces.Box(-np.inf, np.inf, (env.n_joints, 3), dtype=np.float32),
#       'orientation': spaces.Box(-np.inf, np.inf, (env.n_joints, 4), dtype=np.float32),
#       'linear_vel': spaces.Box(-np.inf, np.inf, (env.n_joints, 3), dtype=np.float32),
#       'angular_vel': spaces.Box(-np.inf, np.inf, (env.n_joints, 3), dtype=np.float32),
#     })


In [25]:
from stable_baselines3 import A2C
import gym 
# model = PPO('MultiInputPolicy', env, verbose=1)
# model.learn(total_timesteps=10)
# env = gym.make('CartPole-v1')

model = A2C('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10)

obs = env.reset()
for i in range(10):
    action, _state = model.predict(obs, deterministic=True)
    print(action)
    obs, reward, done, info = env.step(action)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
[300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.]
[300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.]
[300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.]
[300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.]
[300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.]
[300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.]
[300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.]
[300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.]
[300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.]
[300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300. 300.]


In [5]:
# missing observation_space and action_space 

In [9]:
from gym import spaces 
_action_space = spaces.Box(0, 2, (2,2), dtype=np.float32)


In [10]:
_action_space

Box([[0. 0.]
 [0. 0.]], [[2. 2.]
 [2. 2.]], (2, 2), float32)